### 환경 설정

In [1]:
!pip3 install -U pip
!pip3 install -U selenium
!pip3 install -U pandas

In [2]:
!pip3 install selenium webdriver_manager
!pip3 install selenium pyperclip

### 모듈 import

In [3]:
import pandas as pd
import pyperclip

# 크롬 드라이버 기본 모듈
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# 크롬 드라이버 자동 업데이트을 위한 모듈
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

# 브라우저 꺼짐 방지 옵션
chrome_options = Options() # Option class 생성
chrome_options.add_experimental_option("detach", True) # add_exp~ 메서드를 이용해서 브라우저와 소스코드의 종료 관계 detach

# 불필요한 에러 메시지 삭제
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"]) 

# 크롬 드라이버 최신 버전 설정
service = Service(executable_path=ChromeDriverManager().install())

browser = webdriver.Chrome(service=service, options=chrome_options)

### 사이트 접속 & 로그인

In [4]:
# 웹페이지 해당 주소 이동
BJ_LOGIN_URL = 'https://www.acmicpc.net/login?next=%2F'
browser.get(BJ_LOGIN_URL)
browser.implicitly_wait(0.5)

# 아이디 비밀번호 저장
ID='~~~~@~~~~.com'
PW='~~~~'

# HTML 태그값 변경 후 click(아이디 비밀번호 입력)
id_input = browser.find_element(By.NAME, "login_user_id")
id_input.click()
id_input.send_keys(ID)
pw_input = browser.find_element(By.NAME, "login_password")
pw_input.click()
pw_input.send_keys(PW)

# 로그인 버튼 클릭
login_btn = browser.find_element(By.CSS_SELECTOR, "#submit_button")
login_btn.click()
browser.implicitly_wait(15) # recaptcha

### 맞은 문제번호 리스트로 저장

In [5]:
# 마이페이지 이동
user_name = browser.find_element(By.CLASS_NAME, "username")
user_name.click()

#맞은 문제 리스트로 저장
problem_list = browser.find_elements(By.CSS_SELECTOR,'body > div.wrapper > div.container.content > div.row > div:nth-child(2) > div > div.col-md-9 > div:nth-child(2) > div.panel-body > div > a')
problem_list=list(map(lambda page:page.get_attribute("innerHTML"),problem_list))
print(problem_list)

['1000', '1001', '1002', '1003', '1004', '1008', '1009', '1010', '1011', '1016', '1018', '1021', '1032', '1033', '1037', '1043', '1065', '1085', '1100', '1110', '1149', '1152', '1157', '1158', '1167', '1181', '1182', '1193', '1197', '1219', '1253', '1259', '1260', '1267', '1269', '1292', '1300', '1316', '1330', '1357', '1377', '1389', '1406', '1427', '1436', '1439', '1449', '1456', '1463', '1475', '1476', '1516', '1517', '1541', '1546', '1550', '1620', '1629', '1654', '1707', '1712', '1715', '1717', '1735', '1744', '1747', '1753', '1759', '1764', '1780', '1789', '1850', '1854', '1874', '1904', '1912', '1916', '1919', '1920', '1924', '1927', '1929', '1931', '1932', '1934', '1940', '1948', '1966', '1967', '1976', '1977', '1978', '1992', '2018', '2023', '2108', '2110', '2156', '2163', '2164', '2167', '2178', '2193', '2212', '2231', '2251', '2252', '2292', '2309', '2343', '2346', '2420', '2438', '2439', '2440', '2441', '2442', '2443', '2444', '2445', '2446', '2447', '2455', '2468', '2475',

### 문제별로 데이터 추출
- 문제 번호
- 문제 제목
- 문제 난이도 https://d2gd6pc034wcta.cloudfront.net/tier/1.svg, (1~5 브론즈5~브론즈1) (6~10 실버5~실버1) ...
- 맞은 코드 제출 내역 (제출 번호)
- 메모리
- 시간
- 프로그래밍 언어
- 코드 길이
- 제출 날짜

In [6]:
submit_list=[]
def getProblemData(problem_number):
    # 문제 URL 주소로 문제 페이지 접속
    browser.get('https://www.acmicpc.net/problem/'+problem_number)
    # 문제 제목
    problem_title = browser.find_element(By.CSS_SELECTOR,'#problem_title').get_attribute("innerText")

    # 문제 레벨
    problem_level_url = browser.find_element(By.CSS_SELECTOR,'body > div.wrapper > div.container.content > div.row > div:nth-child(2) > ul > li.active > a > img').get_attribute('src')
    problem_level = problem_level_url.split('/')[-1].split('.')[0]

    # 내 제출 내역으로 이동
    my_submit = browser.find_element(By.CSS_SELECTOR,'body > div.wrapper > div.container.content > div.row > div:nth-child(2) > ul > li:nth-child(7) > a')
    my_submit.click()

    # 내 제출 내역 중 맞은 내역만 리스트로 전환
    my_submit_list = browser.find_elements(By.CSS_SELECTOR, '#status-table > tbody > tr')
    my_submit_list = list(filter(lambda submit:submit.find_element(By.CLASS_NAME,'result').get_attribute("innerText") == '맞았습니다!!',my_submit_list))
    
    # 리스트 순회, 내 정답 데이터 추출
    for submit in my_submit_list:
        submit_id = submit.find_element(By.CSS_SELECTOR,'td:nth-child(1)').get_attribute('innerHTML')

        memory = submit.find_element(By.CLASS_NAME,'memory').get_attribute('innerHTML').replace('<span class="','').replace('-text"></span>','')

        time = submit.find_element(By.CLASS_NAME,'time').get_attribute('innerHTML').replace('<span class="','').replace('-text"></span>','')

        language = submit.find_element(By.CSS_SELECTOR,'td:nth-child(7) > a').get_attribute('innerHTML')

        code_length = submit.find_element(By.CSS_SELECTOR,'td:nth-child(8)').get_attribute('innerHTML').replace('<span class="','').replace('-text"></span>','')

        submit_date = submit.find_element(By.CSS_SELECTOR,'td:nth-child(9) > a').get_attribute('data-original-title')
    
        # 정답 데이터 저장
        submit_list.append([problem_number,problem_title,problem_level,submit_id,memory,time,language,code_length,submit_date])
    

### 문제별로 풀이내역 저장

In [7]:
for problem in problem_list:
    getProblemData(problem)

In [8]:
submit_df=pd.DataFrame(submit_list,columns=['문제번호','문제제목','문제난이도','제출번호','메모리','시간','언어','코드길이','제출날짜'])
submit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581 entries, 0 to 580
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   문제번호    581 non-null    object
 1   문제제목    581 non-null    object
 2   문제난이도   581 non-null    object
 3   제출번호    581 non-null    object
 4   메모리     581 non-null    object
 5   시간      581 non-null    object
 6   언어      581 non-null    object
 7   코드길이    581 non-null    object
 8   제출날짜    581 non-null    object
dtypes: object(9)
memory usage: 41.0+ KB


In [9]:
solvedCode_list=[]
for code_id in submit_df['제출번호']:
    CODE_URL='https://www.acmicpc.net/source/'+code_id
    browser.get(CODE_URL)
    paste_button = browser.find_element(By.CSS_SELECTOR,'body > div.wrapper > div.container.content > div.row > div.col-md-3 > div > button')
    paste_button.click() # 복사
    code = pyperclip.paste() # 붙여넣기
    solvedCode_list.append([code_id,code])

solvedCode_df = pd.DataFrame(solvedCode_list,columns=['제출번호','제출코드'])
solvedCode_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581 entries, 0 to 580
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제출번호    581 non-null    object
 1   제출코드    581 non-null    object
dtypes: object(2)
memory usage: 9.2+ KB


In [10]:
solvedCode_df['제출코드'][10]

'import sys\ninput=sys.stdin.readline\nn=int(input())\nfor _ in range(n):\n    a,b=map(int,input().split())\n    v=pow(a,b,10)\n    print(v if v!=0 else 10)'

### 두 데이터프레임 조인, 언어별로 최신 코드만 저장
- on 제출코드

In [64]:
join_table=pd.merge(submit_df,solvedCode_df)
join_table=join_table.groupby(['문제번호','언어']).first().reset_index()
join_table.info()

### 소스코드 파일로 전환

In [73]:
lang={'C99':'c','Python 3':'py','PyPy3':'py','Java 11':'java','C++17':'cpp'}
level=['','B5','B4','B3','B2','B1','S5','S4','S3','S2','S1','G5','G4','G3','G2','G1','P5','P4','P3','P2','P1','D5','D4','D3','D2','D1']
full_level={'B':'Bronze','S':'Silver','G':'Gold','P':'Platinum','D':'Diamond'}
for i in range(len(join_table)): # join_table로 변경
    tuple=join_table.iloc[i]
    with open('BJ/'+full_level[level[int(tuple['문제난이도'])][0]]+'/'+level[int(tuple['문제난이도'])]+'_'+tuple['문제번호']+'.'+lang[tuple['언어']], 'w') as file:
        file.write(
            '# ' + tuple['문제제목'] + ', ' + tuple['제출날짜'] + ', ' + tuple['메모리'] + ', ' + tuple['시간'] + ', ' + tuple['코드길이'] + '\n'
            + tuple['제출코드'] + '\n')